Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

<strong>Part A - Answer</strong><br><br>
Let $E$ be the event that the defendant is guilty <br>
Let $X$ be the event that the blood type is a match

Here, the prosecutor claims $P(X)=P(E|X)$

However, according to Bayes, $P(E|X) = \frac{P(X|E)P(E)}{P(X)}$

* $P(E)$ is the prior evidence that a person is guilty which depends on how many people had access to the crime scene. 

* $P(X|E)$ is the likelihood that the blood type is a match given that the defendant is guilty (in this case, for the example I give I'm going to assume this is 100%)

* $P(X)$ is the probability that any given person has the rare blood type

The prosecutor mistakenly concludes that the odds of a person with a matching blood type is equal to the probability that an individual person might have the rare blood type. But, how can we know that there weren't hundreds of people exposed to the crime scene of which there were a several individuals who had the rare blood type? If we change our thinking, then we can see the flaw in our logic. If we knew 800 people had been exposed to the scene, then we could conclude that 8 of them had the matching blood type. If we assume our likelihood $P(X|E) = 1$, then to solve $P(E|X$ we would calculate $\frac{1 * \frac{1}{.01 * 800}}{.01}$ which would mean that there really is only a $12.5$% chance that the defendant is guilty.

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument?

<strong>Part B - Answer</strong><br><br>

Let $E$ be the event that the defendant is guilty <br>
Let $X$ be the event that the blood type is a match

* $P(E)$ is the prior evidence that a person is guilty which depends on how many people had access to the crime scene. 

* $P(X|E)$ is the likelihood that the blood type is a match given that the defendant is guilty (I'm going to assume this is 100% for simplicity in my arguments)

* $P(X)$ is the probability that any given person has the rare blood type

Here, the defender claims the posterior probability is the same as the prior probability, $P(E|X)=P(E)$ (the probability of a person being guilty given that they have matching blood types is equal to the prior probability that the person is guilty). However, if we look at Bayes thorem, we can obviously see that the defender fails to take into account the probability that the blood type is a match, ${P(X)}$, in the denominator of the equation.

$P(E|X) = \frac{P(X|E)P(E)}{P(X)}$

If the defender divided the result of $\frac{1}{8000}$ by $P(X)$, he would have had the right answer.


c. Suppose that forensic analysis tells us that with 98% probability, the blood found at the scene belongs to the perpetrator.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene.  

<strong>Part C - Answer</strong>
<br><br>
*Bayes Theorem* 
<br><br>
$P(E|X) = \frac{P(X|E)P(E)}{P(X)}$
<br><br>
We want to find the posterior probability ${P(E|X)}$
<br><br>
We know:
* likelihood - $P(X|E) = .98$
* prior - $P(X) = \frac{1}{5}$
* $P(X) = .01$

Plugging these values into Bayes Theorem we get:

$P(E|X) = \frac{.98 * \frac{1}{5}}{.01}$

$P(E|X) = 19.6$%

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|\hat{X}) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|\hat{X}) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [115]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam

word_dictionary = {}
# this is an example of what the work dictionary will eventually look like
# { sale: [(# times word shows up in span message), (#times word shows up in ham message)]}
# minimum of count of a word appearing is 1, which eliminates the difficulty of dividing by 0
#     this is also known as regularization, prevents one word from having too much influence

training_labels = np.zeros(len(lines),dtype=int) # this is just an array of 0's [0,0,0,....0,0,0]

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1] # regularize, set both ham and spam values equal to 1 to start off with
        word_dictionary[w][mapper(l[0])] += 1 # if it's spam increment first value in array, if it's ham increment second value in array
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [121]:
#What is the prior P(Y=ham) ?
numberOfHamMessages = np.count_nonzero(training_labels == 1) # number of times ham message shows up
numberOfSpamMessages = np.count_nonzero(training_labels == 0) # number of spam messages
totalNumberOfMessages = training_labels.size # total number of messages

ham_prior = numberOfHamMessages/totalNumberOfMessages # None!

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = (val[1] / (val[0] + val[1])) # None!

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [122]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0 
    posterior_spam = 1.0 
    #! Don't forget to include the prior!
    # Loop over all the words in each message
    for w in m:
        # #! What is the purpose of this try/except handler? 
        # ANSWER: There may be words in the test data that we didn't see in the training data
        try:
            posterior_ham *= (ham_prior * ham_likelihood[w]) #!
            posterior_spam *= ((1-ham_prior) * (1-ham_likelihood[w])) #! <
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [124]:
# We need to make an assumption about whether to classify any given word in our posteriors as spam or ham
# so I am creating a method that accepts two parameters and returns ham/spam based on which posterior probability
# is greater.
def hamOrSpam(posteriorSpam, posteriorHam):
    if(posteriorSpam > posteriorHam):
        return mapper('spam')
    elif(posteriorHam >= posteriorSpam):
        return mapper('ham')

# now we need to loop back through the test data and compare our models prediction to what the actual label for the 
# message is. 
numberCorrect = 0
numberWrong = 0

for i,line in enumerate(test_messages):
    if(mapper(line[0]) == hamOrSpam(posteriors[i][0], posteriors[i][1])):
        numberCorrect += 1
    else:
        numberWrong += 1
        
classifierAccuracy = (numberCorrect / (numberCorrect + numberWrong)) * 100
print('Classifier Accuracy: ' + str(classifierAccuracy))

Classifier Accuracy: 84.91921005385996


<h1 style="text-decoration: underline; color: #333">Summary</h1>
<h4 style="color: #333;">Assumptions</h4>
I assumed that a message was ham/spam depending on which posterior probability was greater. The logic for this is in the method hamOrSpam() that I created.
<br>
<h4 style="color: #333;">Conclusion</h4>
Our model is 84.92% accurate in predicting the correct category for our test data set.